In [ ]:
#drive.mount('/content/drive')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

     |████████████████████████████████| 3.4 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 36.3 MB/s 
     |████████████████████████████████| 895 kB 50.9 MB/s 
     |████████████████████████████████| 61 kB 495 kB/s 
     |████████████████████████████████| 3.3 MB 31.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [24]:
import pandas as pd
import numpy as np
import transformers
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
import json
from tqdm import tqdm
from torch.nn.functional import one_hot
from sklearn.metrics import f1_score

#Data preprocess

In [ ]:
# xquad_vi_df = pd.read_json('/content/drive/MyDrive/Fine-Tune-Bert/data/_train_xquad_vi.json')

In [ ]:
# mlqa_df = pd.read_json('/content/drive/MyDrive/Fine-Tune-Bert/data/mlqa_trainingdata.json')

In [ ]:
# # convert mlqa format to zalo format
# datas = []
# i = 0
# for data in mlqa_df['data']:
#   for paragraph in data['paragraphs']:
#     for qas in paragraph['qas']:
#       temp_dict = {}
#       temp_dict['id'] = qas['id']
#       temp_dict['question'] = qas['question']
#       temp_dict['title'] = data['title']
#       temp_dict['context'] = paragraph['context']
#       temp_dict['label'] = True
#       i+=1
#       print(i)
#       datas.append(temp_dict)

In [ ]:
# with open('/content/mlqa_vi_zalo_format.json', 'w',encoding='utf-8') as outfile:
#      json.dump(datas, outfile,ensure_ascii=False,indent=4)

In [ ]:
# convert xquad format to zalo format
# xquad_vi_dict = {}
# datas = []
# i = 0
# for data in xquad_vi_df['data']:
#   for paragraph in data['paragraphs']:
#     for qas in paragraph['qas']:
#       temp_dict = {}
#       temp_dict['id'] = qas['id']
#       temp_dict['question'] = qas['question']
#       temp_dict['title'] = data['title']
#       temp_dict['context'] = paragraph['context']
#       temp_dict['label'] = True
      
#       datas.append(temp_dict)

In [ ]:
# with open('/content/xquad_vi_zalo_format.json', 'w',encoding='utf-8') as outfile:
#     json.dump(datas, outfile,ensure_ascii=False,indent=4)

In [ ]:
# zalodata_df = pd.read_json('/content/drive/MyDrive/Fine-Tune-Bert/data/zalo_dataset.json')

In [ ]:
# zalodata_df.shape

In [ ]:
# chia tập test lấy từ bộ dữ liệu của zalo
# with open('/content/drive/MyDrive/Fine-Tune-Bert/data/testing_zalodata.json', 'w') as file:
#     zalodata_df[0:3000].to_json(file, orient='records',force_ascii=False,indent=4)

In [ ]:
# with open('/content/drive/MyDrive/Fine-Tune-Bert/data/trainingdata_zalo.json', 'w') as file:
#     zalodata_df[3000:].to_json(file, orient='records',force_ascii=False,indent=4)

#Train

In [50]:
# mlqadata_df = pd.read_json('/content/drive/MyDrive/Fine-Tune-Bert/data/mlqa_vi_zalo_format.json')
# xquaddata_df = pd.read_json('/content/drive/MyDrive/Fine-Tune-Bert/data/xquad_vi_zaloformat_training_data.json')
zalodata_df = pd.read_json('/content/drive/MyDrive/Fine-Tune-Bert/data/trainingdata_zalo.json')

In [51]:
zalodata_df = zalodata_df.rename({'text':'context'},axis =1)

In [49]:
# trainingdata = pd.concat((zalodata_df,mlqadata_df,xquaddata_df))

In [7]:
# trainingdata.shape

(15859, 5)

In [8]:
# cuda = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [52]:
columns = ["context", 'question','label']
data = []
for index, row in zalodata_df.iterrows():
  temp = []
  temp.append(row['context'])
  temp.append(row['question'])
  temp.append(int(row['label']))
  data.append(temp)

In [53]:
phobert = AutoModel.from_pretrained("vinai/phobert-base",output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

In [54]:
data = pd.DataFrame(data, columns=columns)

In [55]:
data = data.sample(frac = 1)

In [56]:
class TextDataset(Dataset):
  def __init__(self, text, question, label, tokenizer, max_length):
    super(TextDataset, self).__init__()
    self.tokenizer = tokenizer
    self.text = text
    self.question = question
    self.label = label
    self.max_length = max_length
  def __len__(self):
    return len(self.label)
  def __getitem__(self, index):
    inputs = self.tokenizer.encode_plus(
        self.text[index],self.question[index],
        pad_to_max_length = True,
        add_special_tokens = True,
        return_attention_mask = True,
        max_length = self.max_length,
        truncation = True,
    )
    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]

    return{
        "ids" : torch.tensor(ids, dtype = torch.long, device = cuda),
        "attention_mask" : torch.tensor(mask, dtype = torch.long, device=cuda),
        "target" : torch.tensor(self.label[index], dtype = torch.long, device=cuda),
    }

In [57]:
dataset = TextDataset(data['context'],data['question'],data['label'],tokenizer, 256)

In [58]:
datatrainloader = DataLoader(dataset=dataset, batch_size=16)

In [59]:
class BERT(nn.Module):
  def __init__(self, bert):
    super(BERT, self).__init__()
    self.bert = bert
    self.linear_1 = nn.Linear(768*4, 128)
    self.linear_2 = nn.Linear(128, 2)
    # self.sigmoid = nn.Sigmoid()
  def forward(self, input_ids, attention_mask):
    output = self.bert(input_ids = input_ids, attention_mask = attention_mask)
    cls_output = torch.cat((output['hidden_states'][-1][:,0,:],
                            output['hidden_states'][-2][:,0,:],
                            output['hidden_states'][-3][:,0,:],
                            output['hidden_states'][-4][:,0,:]), dim = -1)
    output = self.linear_1(cls_output)
    output = self.linear_2(output)
    # output = self.sigmoid(output)
    return output

In [60]:
model = BERT(phobert).to(cuda)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-6)

In [61]:
for param in model.bert.parameters():
  param.requires_grad = False

In [62]:
transformers.logging.set_verbosity_error()

In [63]:
model.train()
for epoch in range(5):
  print(epoch)
  loop = tqdm(enumerate(datatrainloader), leave=False, total=len(datatrainloader))
  for batch, dl in loop:
    ids = dl['ids']
    attention_mask = dl['attention_mask']
    label = dl['target']

    optimizer.zero_grad()
    output = model(input_ids=ids, attention_mask = attention_mask)
    # print(output)
    # print(label)
    # temp = torch.tensor(one_hot(label, 2),dtype=torch.float, device=cuda)
    loss = loss_function(output, label)

    predict =torch.argmax(output, dim = 1)
    correct = sum(1 for a, b in zip(predict, label) if a == b)
    samples = output.shape[0]
    accuracy = correct/samples
    loss.backward()
    optimizer.step()

    loop.set_description(f'Epoch={epoch}/10')
    loop.set_postfix(loss=loss.item(),acc=accuracy)

0


  0%|          | 0/945 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


1


2


3


4


In [64]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Fine-Tune-Bert/model_QA_1CLS_phobert_zalo_5epoch.pth')

In [ ]:
# model = BERT(phobert).to(cuda)
# model.load_state_dict(torch.load('/content/drive/MyDrive/Fine-Tune-Bert/model_QA_4CLS_phobert.pth'))

In [65]:
def accuracy(datatrainloader,model):
  loop = tqdm(enumerate(datatrainloader), leave=False, total=len(datatrainloader))
  samples = 0
  correct = 0
  for batch, dl in loop:
    ids = dl['ids']
    attention_mask = dl['attention_mask']
    label = dl['target']

    output = model(input_ids=ids, attention_mask = attention_mask)
    predict =torch.argmax(output, dim = 1)

    temp_correct = sum(1 for a, b in zip(predict, label) if a == b)
    temp_samples = output.shape[0]
    samples += temp_samples
    correct += temp_correct
    accuracy = temp_correct/temp_samples

    loop.set_postfix(acc=accuracy)
  return correct*1.0/samples

In [66]:
print(accuracy(datatrainloader,model))

  0%|          | 0/945 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
                                                           

0.701085517606566


#Evaluate in Test Set

In [68]:
model = BERT(phobert).to(cuda)
model.load_state_dict(torch.load('/content/drive/MyDrive/Fine-Tune-Bert/model_QA_1CLS_phobert_zalo_5epoch.pth'))

<All keys matched successfully>

In [69]:
testset = pd.read_json('/content/drive/MyDrive/Fine-Tune-Bert/data/testing_zalodata.json')

In [70]:
testset.shape

(3000, 5)

In [71]:
testset = testset.rename({'text':'context'}, axis = 1)

In [72]:
columns = ["context", 'question','label']
data = []
for index, row in testset.iterrows():
  temp = []
  temp.append(row['context'])
  temp.append(row['question'])
  temp.append(int(row['label']))
  data.append(temp)
data = pd.DataFrame(data, columns=columns)

In [76]:
tx = TextDataset(data['context'][0:3000],data['question'][0:3000],data['label'][0:3000],tokenizer, 256)

In [77]:
dl = DataLoader(dataset=tx, batch_size=128)

In [78]:
print(accuracy(dl,model))

  0%|          | 0/24 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
                                                          

0.593


#Demo

In [ ]:
line = "Tôi là sinh_viên trường đại_học Công_nghệ ."

input_ids = torch.tensor([tokenizer.encode(line)])

with torch.no_grad():
    features = phobert(input_ids)  # Models outputs are now tuples


In [ ]:
features.keys()

In [ ]:
features['hidden_states'][-1][:,0,:].shape

In [ ]:
torch.cat((features['hidden_states'][-1][:,0,:],
           features['hidden_states'][-2][:,0,:],
           features['hidden_states'][-3][:,0,:],
           features['hidden_states'][-4][:,0,:]), dim = -1).shape

In [ ]:
features.keys()

In [ ]:
features['hidden_states'][-1][:,0,:].shape

In [ ]:
torch.cat((features['hidden_states'][-1][:,0,:],
           features['hidden_states'][-2][:,0,:],
           features['hidden_states'][-3][:,0,:],
           features['hidden_states'][-4][:,0,:]), dim = -1).shape